In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
from tensorflow import keras

from lib.utils import *
from src.config import c
from src.data_utils import *
from src.generator import Generator
from src.geo_filter import filters as geo_filters
from src.kaggle_utils import predict
from src.models import custom_objects as models_custom_objects
from src.services import *

In [18]:
fix_random_seed(c["SEED"])

In [19]:
DATASET = "/app/_work/dataset-D.pickle"
VAL_SPLIT = 0.25
MODEL = "/app/_work/kaggle_models/D1_xae332.h5"

In [64]:
df = pd.read_pickle(DATASET)
df = df[df._source == "long"]

In [66]:
len(df)

2848

In [60]:
filenames = list(df["filename"].unique())
np.random.shuffle(filenames)
filenames_tr = filenames[: int((1 - VAL_SPLIT) * len(filenames))]
filenames_val = filenames[len(filenames_tr) :]

In [61]:
df_tr = df.query("filename in @filenames_tr")
df_val = df.query("filename in @filenames_val")
len(df_tr), len(df_val)

(2183, 665)

In [62]:
set(list(df_tr['filename'].unique())) == set(filenames_tr)

True

In [63]:
set(list(df_val['filename'].unique())) == set(filenames_val)

True

In [7]:
meta = read_json(MODEL.replace(".h5", ".json"))

In [8]:
model = keras.models.load_model(MODEL, custom_objects=models_custom_objects)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA Tesla V100-DGXS-32GB, compute capability 7.0


In [19]:
features_o = model.get_layer("features").get_output_at(0)

In [22]:
model2 = keras.models.Model(inputs=model.inputs, outputs=[features_o] + model.outputs)